pip install ultralytics

pip install xmltodict

1. conda create --prefix "C:\\Users\\rs659\\Desktop\\Object-Detection\\wincondaprojenv" python=3.11.6  
2. conda activate "C:\\Users\\rs659\\Desktop\\Object-Detection\\wincondaprojenv"
3. pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
4. conda install -p c:\Users\rs659\Desktop\Object-Detection\wincondaprojenv ipykernel --update-deps --force-reinstall


pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
conda install -p c:\Users\rs659\Desktop\Object-Detection\wincondaprojenv ipykernel --update-deps --force-reinstall

5. pip install pybind11
6. pip install opencv-python
7. pip instal ultralytics
8. pip install pylabel




Merge Different COCO dataset
https://gradiant.github.io/pyodi/reference/apps/coco-merge/
pyodi coco merge coco_1.json coco_2.json output.json


In [ ]:
import os
import zipfile
from pylabel import importer

In [ ]:
#  'DatasetDetectron2\\annotations\\MergeDataset\\train\\05993C52-D3D7-4847-8CB3-8AC92F9586EF_1_105_c.jpeg'

### FOR TRAIN ###
#Specify path to the coco.json file
path_to_annotations = "DatasetDetectron2/annotations/train.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = os.getcwd() + "/MergeDataset/train/"

#Import the dataset into the pylable schema
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df.head(5)

dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.2, test_pct=.2)
dataset.analyze.ShowClassSplits()
# dataset.export.ExportToYoloV5(output_path='YOLO_dataset/train/labels',yaml_file='YOLO_dataset/train/dataset.yaml', copy_images=True, use_splits=True)

dataset.export.ExportToYoloV5(output_path='YOLO_dataset_train/labels',yaml_file='dataset.yaml', copy_images=True)

In [ ]:
### FOR TEST ###

# #Specify path to the coco.json file
path_to_annotations = "DatasetDetectron2/annotations/test.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = os.getcwd() + "/MergeDataset/train/"

#Import the dataset into the pylable schema
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df.head(5)

dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.2, test_pct=.2)
dataset.analyze.ShowClassSplits()
# dataset.export.ExportToYoloV5(output_path='YOLO_dataset/train/labels',yaml_file='YOLO_dataset/train/dataset.yaml', copy_images=True, use_splits=True)

dataset.export.ExportToYoloV5(output_path='YOLO_dataset_test/labels',yaml_file='dataset.yaml', copy_images=True)


In [ ]:
import os

# Create the main directory
os.makedirs('YOLO_dataset', exist_ok=True)

# # Create subdirectories for labels and images
# os.makedirs('YOLO_dataset/labels/train', exist_ok=True)
# os.makedirs('YOLO_dataset/labels/test', exist_ok=True)
# os.makedirs('YOLO_dataset/images/train', exist_ok=True)
# os.makedirs('YOLO_dataset/images/test', exist_ok=True)

In [ ]:
# !cp YOLO_dataset_test/images YOLO_dataset\images\train
# !cp YOLO_dataset_train/images YOLO_dataset\images\test

import shutil

# Copy images from YOLO_dataset_test/images to YOLO_dataset/images/train
shutil.copytree('YOLO_dataset_test/images', 'YOLO_dataset/images/train')

# Copy images from YOLO_dataset_train/images to YOLO_dataset/images/test
shutil.copytree('YOLO_dataset_train/images', 'YOLO_dataset/images/test')

In [ ]:
# !copy YOLO_dataset_test/labels YOLO_dataset\labels
# !copy YOLO_dataset_train/labels YOLO_dataset\labels

# shutil.copytree('YOLO_dataset_test/labels', 'YOLO_dataset/labels')

# shutil.copytree('YOLO_dataset_train/labels', 'YOLO_dataset/labels/')

# Define source and destination directories
# source_dirs = ['YOLO_dataset_test/labels', 'YOLO_dataset_train/labels']
# destination_dir = 'YOLO_dataset/labels'

# # Copy contents of the first directory
# for source_dir in source_dirs:
#     for root, dirs, files in os.walk(source_dir):
#         for file in files:
#             source_file = os.path.join(root, file)
#             relative_path = os.path.relpath(source_file, source_dir)
#             destination_file = os.path.join(destination_dir, relative_path)
#             os.makedirs(os.path.dirname(destination_file), exist_ok=True)
#             shutil.copy(source_file, destination_file)


import shutil
import os

# Define source and destination paths
test_source = 'YOLO_dataset_test/labels'
train_source = 'YOLO_dataset_train/labels'
test_dest = 'YOLO_dataset/labels/test'
train_dest = 'YOLO_dataset/labels/train'

# # Ensure destination directories exist
# os.makedirs(test_dest, exist_ok=True)
# os.makedirs(train_dest, exist_ok=True)

# Copy directories
try:
    shutil.copytree(test_source, test_dest)
    shutil.copytree(train_source, train_dest)
    print("Directories copied successfully!")
except FileExistsError:
    print("Destination directories already exist. Remove or rename them and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# !copy DatasetDetectron2\annotations\dataset.yaml YOLO_dataset\

source_file = "DatasetDetectron2/annotations/dataset.yaml"
destination_dir = "YOLO_dataset"

shutil.copy(source_file, destination_dir)

In [ ]:
# https://github.com/pylabel-project/samples/blob/main/coco2yolov5.ipynb
# dataset.path_to_annotations = "data/yolo"
# dataset.export.ExportToYoloV5()[0]

In [ ]:
import os
from ultralytics import YOLO


# ROOT_DIR = 'C:\\Users\\rs659\\Desktop\\Object-Detection\\YOLO_dataset'
# ROOT_DIR = 'C:\\Users\\rs659\\Desktop\\Object-Detection//trainingsplit//'
ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset'


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('yolov8n.pt')


# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=1)  # train the model 

In [ ]:
import os

from ultralytics import YOLO

# ROOT_DIR = 'C:\\Users\\rs659\\Desktop\\Object-Detection\\YOLO_dataset\\'
ROOT_DIR = 'C:\\Users\\rs659\\Desktop\\Object-Detection//trainingsplit//'
# ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Project2/training'


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('yolov8n.pt')

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=2, save_dir='models/YOLO')  # train the model 